In [1]:
import numpy as np
import pickle

# Function to adaptively discretize deltas
def adaptive_discretize(delta):
    # Small changes
    if -0.5 <= delta <= 0.5:
        return 0
    elif -1 <= delta < -0.5:
        return -1
    elif 0.5 < delta <= 1:
        return 1
    # Large changes
    elif delta > 1 and delta <= 3:
        return 3
    elif delta > 3:
        return 5
    elif delta < -1 and delta >= -3:
        return -3
    elif delta < -3:
        return -5

# Function to recreate dataset with adaptive discretization
def recreate_dataset_with_adaptive_discretization(original_data, delta_data):
    recreated_data = {}  # Dictionary to store the recreated dataset

    for video_key, video_data in original_data.items():
        print(f"Processing video: {video_key}")

        # Extract the original keypoints and delta values
        original_keypoints = video_data['keypoints']  # Shape: (frames, points, 3)
        delta_keypoints = delta_data[video_key]['keypoints']  # Shape: (frames, points, 3)

        # Initialize the recreated dataset
        frames, points, features = original_keypoints.shape
        recreated_frames = np.zeros((frames, points, features), dtype=np.float32)

        # Use the first frame's keypoints as the starting frame (converted to integers)
        recreated_frames[0, :, :2] = np.round(original_keypoints[0, :, :2]).astype(np.int32)  # X, Y as integers
        recreated_frames[0, :, 2] = np.round(original_keypoints[0, :, 2], decimals=2)  # Confidence as float (2 digits)

        # Iteratively add delta values to recreate the dataset with adaptive discretization
        for i in range(1, frames):
            for p in range(points):
                # Apply adaptive discretization to deltas
                discrete_x = adaptive_discretize(delta_keypoints[i, p, 0])
                discrete_y = adaptive_discretize(delta_keypoints[i, p, 1])

                # Update recreated frame values
                recreated_frames[i, p, 0] = np.round(recreated_frames[i - 1, p, 0] + discrete_x).astype(np.int32)  # X as int
                recreated_frames[i, p, 1] = np.round(recreated_frames[i - 1, p, 1] + discrete_y).astype(np.int32)  # Y as int
                recreated_frames[i, p, 2] = np.round(delta_keypoints[i, p, 2], decimals=2)  # Confidence (2 digits)

        # Store the recreated data
        recreated_data[video_key] = {'keypoints': recreated_frames}

        # break  # Uncomment this to process all videos instead of one

    return recreated_data

# Load the datasets
def load_pkl(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

original_data_path = '/nas/Dataset/Phoenix/phoenix-2014-keypoints.pkl'
delta_data_path = '/nas/Chingiz/sing_language/delta/phoenix_deltas_all.pkl'

# Load the original data and delta data
original_data = load_pkl(original_data_path)
delta_data = load_pkl(delta_data_path)

# Recreate the dataset using the delta data
recreated_data = recreate_dataset_with_adaptive_discretization(original_data, delta_data)

# Save the recreated dataset
output_path = '/nas/Chingiz/sing_language/vector/multi_level_disc_with_orgPoint_int.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(recreated_data, f)

print(f"Recreated dataset saved to: {output_path}")


Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-0/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-2/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-3/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-4/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-6/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-7/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_heute_default-8/1/01April_2010_Thursday_heute
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_tagesschau_default-0/1/01April_2010_Thursday_tagesschau
Processing video: fullFrame-210x260px/train/01April_2010_Thursday_tagesschau_default-2/1/01Apr